In [4]:
from config import API_Key
import pandas as pd
import requests
import json
from citipy import citipy
import numpy as np

In [5]:
url = f"http://api.airvisual.com/v2/city?city=Los Angeles&state=California&country=USA&key={API_Key}"
response=requests.get(url)
data=response.json()
print(json.dumps(data,indent=4,sort_keys=4))

{
    "data": {
        "city": "Los Angeles",
        "country": "USA",
        "current": {
            "pollution": {
                "aqicn": 47,
                "aqius": 43,
                "maincn": "p1",
                "mainus": "p1",
                "ts": "2022-06-21T08:00:00.000Z"
            },
            "weather": {
                "hu": 56,
                "ic": "01n",
                "pr": 1011,
                "tp": 20,
                "ts": "2022-06-21T08:00:00.000Z",
                "wd": 100,
                "ws": 2.57
            }
        },
        "location": {
            "coordinates": [
                -118.2417,
                34.0669
            ],
            "type": "Point"
        },
        "state": "California"
    },
    "status": "success"
}


In [6]:
#call climate classifications from csv by lat/lon
filepath="Climate_Data/ClimateClassifications.csv"
climate_df=pd.read_csv(filepath)
climate_df

,Lat,Lon,Cls
0,-89.75,-179.75,EF
1,-89.75,-179.25,EF
2,-89.75,-178.75,EF
3,-89.75,-178.25,EF
4,-89.75,-177.75,EF
...,...,...,...
92411,83.75,-30.75,ET
92412,83.75,-30.25,ET
92413,83.75,-29.75,ET
92414,83.75,-29.25,ET


In [7]:
#create list of nearest city to each row of climate datadrame
nearest_city=[]
nearest_city_df=climate_df
for i in range(0,len(nearest_city_df)):
    lat=nearest_city_df.loc[i,"Lat"]
    lon=nearest_city_df.loc[i,"Lon"]
    city=citipy.nearest_city(lat,lon)
    nearest_city.append(city.city_name)
    
    


In [8]:
#create new dataframe with column showing nearest city to each lat/lon
climate_cities=nearest_city_df
climate_cities["Nearest City"]=nearest_city
climate_cities
climate_cities.to_csv("Climate_Data/Nearest_Cities.csv",index=False)
climate_cities

,Lat,Lon,Cls,Nearest City
0,-89.75,-179.75,EF,vaini
1,-89.75,-179.25,EF,vaini
2,-89.75,-178.75,EF,vaini
3,-89.75,-178.25,EF,vaini
4,-89.75,-177.75,EF,vaini
...,...,...,...,...
92411,83.75,-30.75,ET,illoqqortoormiut
92412,83.75,-30.25,ET,illoqqortoormiut
92413,83.75,-29.75,ET,illoqqortoormiut
92414,83.75,-29.25,ET,illoqqortoormiut


In [9]:
#Remove all duplicate city names to be left with a list of cities with their climate classifcation
cities_shortlist_df=climate_cities.drop_duplicates(subset=["Nearest City"],keep='first')
cities_shortlist_df=cities_shortlist_df.drop(columns=["Lat","Lon"])
cities_shortlist_df=cities_shortlist_df.reset_index(drop=True)
cities_shortlist_df=cities_shortlist_df.rename(columns={"Nearest City":"City","Cls":"Climate"})
cities_shortlist_df=cities_shortlist_df[["City","Climate"]]
cities_shortlist_df.to_csv("Climate_Data/Nearest_Cities_Shortlist.csv",index=False)
cities_shortlist_df

,City,Climate
0,vaini,EF
1,mataura,EF
2,rikitea,EF
3,punta arenas,EF
4,ushuaia,EF
...,...,...
14939,dikson,ET
14940,qaanaaq,ET
14941,longyearbyen,ET
14942,narsaq,ET


In [10]:
world_cities_df=pd.read_csv("Climate_Data/world_cities.csv")
cols = world_cities_df.select_dtypes(include=[object]).columns
world_cities_df[cols] = world_cities_df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
world_cities_df['name']=world_cities_df['name'].str.lower()
world_cities_df.head(25)

,name,country,subcountry,geonameid
0,les escaldes,Andorra,Escaldes-Engordany,3040051
1,andorra la vella,Andorra,Andorra la Vella,3041563
2,umm al qaywayn,United Arab Emirates,Umm al Qaywayn,290594
3,ras al-khaimah,United Arab Emirates,Ras al Khaymah,291074
4,khawr fakkan,United Arab Emirates,Ash Shariqah,291696
5,dubai,United Arab Emirates,Dubai,292223
6,dibba al-fujairah,United Arab Emirates,Al Fujayrah,292231
7,dibba al-hisn,United Arab Emirates,Al Fujayrah,292239
8,sharjah,United Arab Emirates,Ash Shariqah,292672
9,ar ruways,United Arab Emirates,Abu Dhabi,292688


In [11]:
cities_shortlist_df=pd.read_csv("Climate_Data/Nearest_Cities_Shortlist.csv")
complete_df=pd.merge(cities_shortlist_df,world_cities_df,left_on='City',right_on='name')
complete_df=complete_df.drop(columns=["name"])
complete_df=complete_df.rename(columns={"City":"city","Climate":"climate"})

complete_df

,city,climate,country,subcountry,geonameid
0,punta arenas,EF,Chile,Magallanes,3874787
1,ushuaia,EF,Argentina,Tierra del Fuego,3833367
2,hermanus,EF,South Africa,Western Cape,3366880
3,kruisfontein,EF,South Africa,Eastern Cape,986717
4,port elizabeth,EF,South Africa,Eastern Cape,964420
...,...,...,...,...,...
7686,severomorsk,Dfc,Russia,Murmansk,496278
7687,norilsk,Dfc,Russia,Krasnoyarskiy,1497337
7688,murmansk,Dfc,Russia,Murmansk,524305
7689,polyarnyy,Dfc,Russia,Murmansk,506763


In [15]:
#population
poppath="Population_Data/data.csv"
pop_df=pd.read_csv(poppath)
pop_df['city'] = pop_df['city'].str.lower()
pop_df

,city,lat,lng,country,population
0,tokyo,35.6850,139.7514,Japan,35676000
1,new york,40.6943,-73.9249,United States,19354922
2,mexico city,19.4424,-99.1310,Mexico,19028000
3,mumbai,19.0170,72.8570,India,18978000
4,são paulo,-23.5587,-46.6250,Brazil,18845000
...,...,...,...,...,...
4655,pa-an,16.8500,97.6167,Burma,50000
4656,puzi,23.4611,120.2419,Taiwan,50000
4657,korogwe,-5.0896,38.5400,Tanzania,50000
4658,kahemba,-7.2829,19.0000,Congo (Kinshasa),50000


In [17]:
complete_df = pd.merge(complete_df, pop_df, on='city', how='inner')
complete_df

,city,climate,country_x,subcountry,geonameid,lat,lng,country_y,population
0,punta arenas,EF,Chile,Magallanes,3874787,-53.1650,-70.9400,Chile,117430
1,ushuaia,EF,Argentina,Tierra del Fuego,3833367,-54.7900,-68.3100,Argentina,58028
2,port elizabeth,EF,South Africa,Eastern Cape,964420,-33.9700,25.6000,South Africa,1021000
3,east london,EF,South Africa,Eastern Cape,1006984,-32.9700,27.8700,South Africa,338627
4,albany,EF,Australia,Western Australia,2077963,42.6664,-73.7987,United States,597270
...,...,...,...,...,...,...,...,...,...
2652,kandalaksha,Dfc,Russia,Murmansk,553190,67.1643,32.4144,Russia,106798
2653,apatity,Dfc,Russia,Murmansk,581357,67.5731,33.3930,Russia,66907
2654,severomorsk,Dfc,Russia,Murmansk,496278,69.0731,33.4231,Russia,53921
2655,norilsk,Dfc,Russia,Krasnoyarskiy,1497337,69.3400,88.2250,Russia,165873
